In [10]:
# train-hmm.py
# test
from collections import defaultdict

train_input_path = "../../test/05-train-input.txt"

# モデル読み込み
transition = defaultdict(lambda: 0)
emission = defaultdict(lambda: 0)
possible_tags = defaultdict(lambda: 0)

with open(train_input_path) as f:
    for line in f:
        # a_X b_Y a_Z
        word_tags = line.split()
        # ['a_X', 'b_Y', 'a_Z']
        previous = "<s>"
        possible_tags[previous] = 1
        for word, tag in [x.split("_") for x in word_tags]:
            transition[f"{previous} {tag}"] += 1
            possible_tags[tag] = 1
            emission[f"{tag} {word}"] += 1
            previous = tag
        transition[f"{previous} </s>"] += 1

defaultdict(<function __main__.<lambda>()>,
            {'X a': 2, 'Y b': 2, 'Z a': 1, 'X c': 1})

In [18]:
# test-hmm.py
import math
from collections import defaultdict

test_input_path = "../../test/05-test-input.txt"
UNKNOWN_RATE = 0.05
N = 1e6

with open(test_input_path) as f:
    for line in f:
        words = line.split()
        l = len(words)
        best_score = defaultdict(lambda: 0)
        best_edge = defaultdict(lambda: None)
        best_score["0 <s>"] = 0
        best_edge["0 <s>"] = None
        for i in range(l - 1):
            for prev_tag in possible_tags.keys():
                for next_tag in possible_tags.keys():
                    if best_score[f"{i} {prev_tag}"] and transition[f"{prev_tag} {next_tag}"]:
                        pt = transition[f"{prev_tag} {next_tag}"]
                        pe = (1 - UNKNOWN_RATE) * emission[f"{next_tag} {words[i]}"] + UNKNOWN_RATE / N
                        score = best_score[f"{i} {prev_tag}"] - math.log(pt, 2) - math.log(pe, 2)
                        if best_score[f"{i+1} {next_tag}"] == 0 or best_score[f"{i+1} {next_tag}"] < score:
                            best_score[f"{i+1} {next_tag}"] = score
                            best_edge[f"{i+1} {next_tag}"] = f"{i} {prev_tag}"
        for tag in possible_tags.keys():
            if transition[f"{tag} </s>"]:
                score = best_score[f"{l} {tag}"] - math.log(transition[f"{tag} </s>"], 2)
                if best_score[f"{l+1} </s>"] == 0 or best_score[f"{l+1} </s>"] < score:
                    best_score[f"{l+1} </s>"] = score
                    best_edge[f"{l+1} </s>"] = f"{l} {tag}"
        print(best_score)
        # TODO

defaultdict(<function <lambda> at 0x7f12800478c8>, {'0 <s>': 0, '0 X': 0, '0 Y': 0, '0 Z': 0, '1 <s>': 0, '1 X': 0, '1 Y': 0, '1 Z': 0, '3 Y': 0, '4 </s>': 0.0, '3 Z': 0})
defaultdict(<function <lambda> at 0x7f1280047bf8>, {'0 <s>': 0, '0 X': 0, '0 Y': 0, '0 Z': 0, '1 <s>': 0, '1 X': 0, '1 Y': 0, '1 Z': 0, '3 Y': 0, '4 </s>': 0.0, '3 Z': 0})


In [2]:
!cat ../../test/05-train-answer.txt | head -10

T <s> X 1.000000
T X X 0.333333
T X Y 0.666667
T Y </s> 0.500000
T Y Z 0.500000
T Z </s> 1.000000
E X a 0.666667
E X c 0.333333
E Y b 1.000000
E Z a 1.000000


In [3]:
!cat ../../test/05-test-answer.txt | head -10

X Y Z
X X Y


In [ ]:
# train-hmm.py



In [ ]:
# test-hmm.py
